In [7]:
%load_ext autoreload
%autoreload 2

import os 
import sys 
import json

home = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
pths = [os.path.join(home, 'astrobot')]

for pth in pths: 
    if pth not in sys.path: sys.path.insert(0, pth)

from astrobot.utils import *
from astrobot.geomdataframe import GeomDataFrame 
from astrobot.r import R
from astrobot import bem_utils
from astrobot import geom_utils
from astrobot import mtx_utils

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import itertools

import ladybug_geometry.geometry3d as geom3
import ladybug_geometry.geometry2d as geom2
from ladybug_geometry_polyskel import polyskel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
simdir_path = os.path.join('simulations', 'sim_0')

In [9]:
# Generate parameters for shoebox, and pickle to account for seed
# just uniform sampling for now, and no random seed

labels = ['wwr', 'theta']
param_mtx = [np.linspace(0.2, 0.9, 5), 
             np.linspace(0, np.pi * 2, 5)]
param_mtx = np.array(list(itertools.product(*param_mtx))).T

# Save
dfparam = pd.DataFrame({label: row for row, label in zip(param_mtx, labels)})
dfparam.to_pickle(os.path.join(simdir_path, 'sample_mtx.pkl'))

In [10]:
# Generate floor and wall geometers
gdf = GeomDataFrame(pd.read_pickle(os.path.join(simdir_path, 'sample_mtx.pkl')))
b = R(gdf)

# Generate the footprint and spaces
b.mod['mod_geom'] = [geom_utils.face_from_params(theta) for theta in b.mod['theta'].values]
spc_geom, spc_mod_idx = geom_utils.split_mod_to_polyskel(b.mod.idx.values, b.mod['mod_geom'].values)
b.spc = GeomDataFrame({'mod_idx': spc_mod_idx, 'spc_geom': spc_geom})

In [11]:
#space_types = bem_utils.program_space_types('2013::MediumOffice')
space_types = [
     '2013::MediumOffice::ClosedOffice',
     '2013::MediumOffice::OpenOffice', 
     '2013::MediumOffice::Conference',
     '2013::MediumOffice::Corridor',
     '2013::MediumOffice::Elec/MechRoom',
     '2013::MediumOffice::Restroom']
occ = [0.67 / 3.0] * 3
oth = [0.33 / 3.0] * 3  
space_wts = np.array(occ + oth)
blended = bem_utils.program_space_types_blend(space_types, space_wts, 'blended_office')

In [13]:
# Make model, assign program, constructions and wwr
mods = [0] * b.mod.shape[0]
for i, (mod_idx, spcs) in enumerate(b.spc.groupby('mod_idx')):
    
    mod_df = b.mod.iloc[mod_idx]
    
    # Make room2ds from spaces
    spc_geoms, spc_idxs = spcs['spc_geom'].tolist(), spcs.idx
    room2ds = bem_utils.face3ds_to_room2ds(spc_geoms, spc_idxs, mod_idx)
    [bem_utils.set_room2d_program_space_type(room, blended) for room in room2ds]
    
    # Make model from room2ds
    model = bem_utils.room2ds_to_model(room2ds, id=str(mod_idx))
    bem_utils.set_model_wwr(model, mod_df['wwr'], mod_df['theta'])
    bem_utils.set_model_construction_set(model)
    
    mods[i] = model

#b.mod['mod'] = mods

In [28]:
# Visualize
poly = np.array([p.to_array() for p in b.mod.mod_geom.values[0].vertices])
poly




array([[-6., -3.,  0.],
       [ 6., -3.,  0.],
       [ 6.,  3.,  0.],
       [-6.,  3.,  0.]])